In [1]:
'''
import boto3
bucket_name = 'dataminds-homeworks'
s3_file_key = 'data_usage_production.parquet'            # e.g. 'folder/myfile.txt'
local_file_path = 'data_usage_production.parquet'        # Local destination

# Create an S3 client (remove `bucket_name` here — not a valid argument for boto3.client)
s3 = boto3.client(
    's3',
    region_name='us-east-1'
)

# Download the file
try:
    s3.download_file(bucket_name, s3_file_key, local_file_path)
    print(f"✅ File downloaded successfully from s3://{bucket_name}/{s3_file_key} to {local_file_path}")
except Exception as e:
    print("❌ Error downloading file:", e)
'''

'\nimport boto3\nbucket_name = \'dataminds-homeworks\'\ns3_file_key = \'data_usage_production.parquet\'            # e.g. \'folder/myfile.txt\'\nlocal_file_path = \'data_usage_production.parquet\'        # Local destination\n\n# Create an S3 client (remove `bucket_name` here — not a valid argument for boto3.client)\ns3 = boto3.client(\n    \'s3\',\n    region_name=\'us-east-1\'\n)\n\n# Download the file\ntry:\n    s3.download_file(bucket_name, s3_file_key, local_file_path)\n    print(f"✅ File downloaded successfully from s3://{bucket_name}/{s3_file_key} to {local_file_path}")\nexcept Exception as e:\n    print("❌ Error downloading file:", e)\n'

In [2]:
import pandas as pd
df = pd.read_parquet('data_usage_production.parquet')
df.set_index('telephone_number', inplace=True)
#df.head()

In [3]:
import numpy as np
X = df.drop('data_compl_usg_local_m1', axis=1)
y = df['data_compl_usg_local_m1']
y = np.log1p(y)


In [4]:
#I will use it in the pipeline
#numeric_features=X.columns[X.dtypes=='int64'].append(X.columns[X.dtypes=='float64'])
numeric_features=['refill_total_m2', 'refill_total_m3',
                  'frequency','recency','tenure','lastrefillamount_m2',
                  'tot_inact_status_days_l1m_m2','refill_total_m4','data_compl_usg_local_m2','data_compl_usg_local_m3','data_compl_usg_local_m4']
categorical_features=X.columns[X.dtypes=='object']
X = X[numeric_features + list(categorical_features)]

In [ ]:
!pip install category_encoders

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from category_encoders import CatBoostEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_regression

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", CatBoostEncoder()),
        ("selector", SelectPercentile(f_regression, percentile=50))  
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("Regressor", RandomForestRegressor(n_estimators=13,random_state=42))
    ]
)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.751


In [9]:
import boto3

# Replace with your actual credentials and info
bucket_name = 'dataminds-homeworks'
s3_file_key = 'vasif-orujzada-fe1.ipynb' 
local_file_path = 'vasif-orujzada-fe1.ipynb'

# Create an S3 client
s3 = boto3.client(
    's3'
)

# Upload the file
try:
    s3.upload_file(local_file_path, bucket_name, s3_file_key)
    print(f"File uploaded successfully to s3://{bucket_name}/{s3_file_key}")
except Exception as e:
    print("Error uploading file:", e)

File uploaded successfully to s3://dataminds-homeworks/vasif-orujzada-fe1.ipynb
